# K-means

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

In [3]:
#The dataset is quite large, and I am getting memory errors on this particular algorithm.
#I will split it randomnly.
df_features = df_features.sample(frac=0.05, random_state=763)

#We need to reset the index so the the algorithms below can loop through the dataframe
df_features = df_features.reset_index()

### Data Prep with t-SNE dimensionality reduction

Elaboration of my dimensionality reduction configuration decision process is given in the Final Dimension Reduction Configuration notebook.

In [15]:
#Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

In [5]:
# Loading the data
X = df_features
y = df_features.is_canceled

# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

X_norm = normalize(X_std)

In [6]:
from sklearn.manifold import TSNE
import time

In [7]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=100, n_iter=300)
tsne_results = tsne.fit_transform(X_std)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 1978 samples in 0.013s...
[t-SNE] Computed neighbors for 1978 samples in 0.320s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1978
[t-SNE] Computed conditional probabilities for sample 1978 / 1978
[t-SNE] Mean sigma: 1.125884
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.464928
[t-SNE] KL divergence after 300 iterations: 0.494126
t-SNE done! Time elapsed: 10.27414608001709 seconds


## K-Means

The best ARI and Silhouette scores were for the 2 cluster solution

In [8]:
#Libraries
from sklearn.cluster import KMeans

In [9]:
#Libraries
from sklearn import datasets
from sklearn import metrics

## The Best Performer

In [10]:
# Defining the k-means
kmeans_cluster = KMeans(n_clusters=2, random_state=163)

# Fit model
%timeit kmeans_cluster.fit(tsne_results)
K_clusters = kmeans_cluster.predict(tsne_results)
labels = kmeans_cluster.predict(tsne_results)

56.8 ms ± 5.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(K_clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

In [12]:
K_clusters = np.array([0 if x == 1 else 1 if x==0 else 2 for x in K_clusters])

pd.crosstab(y, K_clusters)

col_0,0,1
is_canceled,,
0,572,894
1,512,0


## Finding the Best Performer

In [13]:
#KMeans function that takes an x, y, and a number to get a range of clusters
def kmeans_ari_silh(range_num, x, y):
    from sklearn.cluster import KMeans
    for num in range(2, range_num, 1):
        pred = KMeans(n_clusters=num).fit_predict(x)
        labels = KMeans(n_clusters=num).fit_predict(x)
        print("ARI for {} cluster k-means: {}".format(num,
                                                      metrics.adjusted_rand_score(y, pred)))
        print("silhouette for {} cluster k-means: {}".format(num,
                                                      metrics.silhouette_score(x, labels, metric='euclidean')))
        print('='*60)

In [14]:
kmeans_ari_silh(11, X_std, y)

ARI for 2 cluster k-means: 0.6754724023467227
silhouette for 2 cluster k-means: 0.25709626983088596
ARI for 3 cluster k-means: 0.3199653761507214
silhouette for 3 cluster k-means: 0.2255289812849606
ARI for 4 cluster k-means: 0.31487420388576004
silhouette for 4 cluster k-means: 0.24970367388667528
ARI for 5 cluster k-means: 0.2196820352667906
silhouette for 5 cluster k-means: 0.2523967258352117
ARI for 6 cluster k-means: 0.1769281575658801
silhouette for 6 cluster k-means: 0.2450204322107017
ARI for 7 cluster k-means: 0.14358565970059586
silhouette for 7 cluster k-means: 0.26147871377272025
ARI for 8 cluster k-means: 0.13843277003535354
silhouette for 8 cluster k-means: 0.263514909003147
ARI for 9 cluster k-means: 0.14849630340024905
silhouette for 9 cluster k-means: 0.26203459355797
ARI for 10 cluster k-means: 0.1311516283560575
silhouette for 10 cluster k-means: 0.259963884106798
